In [4]:
import pandas as pd
import numpy as np

# Step 1: Load the TSV files
df1 = pd.read_csv("estat_tec00118.tsv", sep="\t")
df2 = pd.read_csv("estat_une_rt_m 3.tsv", sep="\t")

# ===============================
# df1_melted (Inflation data)
# ===============================

# Step 2: Split the first column into separate variables
df1_split = df1['freq,unit,coicop,geo\\TIME_PERIOD'].str.split(',', expand=True)
df1_split.columns = ['freq', 'unit', 'coicop', 'geo']

# Step 3: Combine with the rest of the data
df1_clean = pd.concat([df1_split, df1.drop(columns=['freq,unit,coicop,geo\\TIME_PERIOD'])], axis=1)

# Step 4: Melt the wide format into long format
df1_melted = df1_clean.melt(
    id_vars=['freq', 'unit', 'coicop', 'geo'],
    var_name='time',
    value_name='inflation'
)

# Step 5: Clean inflation values
df1_melted['inflation'] = df1_melted['inflation'].replace(':', np.nan)
df1_melted['inflation'] = df1_melted['inflation'].str.extract(r'([-+]?\d*\.\d+|\d+)')
df1_melted['inflation'] = df1_melted['inflation'].astype(float)

# ===============================
# df2_melted (Unemployment data)
# ===============================

# Step 6: Split the first column into separate variables
df2_split = df2['freq,s_adj,age,unit,sex,geo\\TIME_PERIOD'].str.split(',', expand=True)
df2_split.columns = ['freq', 's_adj', 'age', 'unit', 'sex', 'geo']

# Step 7: Combine with the rest of the data
df2_clean = pd.concat([df2_split, df2.drop(columns=['freq,s_adj,age,unit,sex,geo\\TIME_PERIOD'])], axis=1)

# Step 8: Melt the wide format into long format
df2_melted = df2_clean.melt(
    id_vars=['freq', 's_adj', 'age', 'unit', 'sex', 'geo'],
    var_name='time',
    value_name='unemployment'
)

# Step 9: Clean unemployment values
df2_melted['unemployment'] = df2_melted['unemployment'].replace(':', np.nan)
df2_melted['unemployment'] = df2_melted['unemployment'].str.extract(r'([-+]?\d*\.\d+|\d+)')
df2_melted['unemployment'] = df2_melted['unemployment'].astype(float)


In [5]:
import pandas as pd

# Step 1: Convert 'time' in unemployment data to datetime, extract year
df2_melted['time'] = pd.to_datetime(df2_melted['time'], errors='coerce')
df2_melted['year'] = df2_melted['time'].dt.year

# Step 2: Drop rows with missing unemployment or invalid dates
df2_cleaned = df2_melted.dropna(subset=['unemployment', 'year', 'geo'])

# Step 3: Aggregate unemployment data by geo and year
unemployment_yearly = (
    df2_cleaned
    .groupby(['geo', 'year'], as_index=False)
    .agg({'unemployment': 'mean'})
)

# Step 4: Convert 'time' column in inflation data to int year
df1_melted['year'] = pd.to_numeric(df1_melted['time'], errors='coerce')
df1_cleaned = df1_melted.dropna(subset=['year', 'geo'])

# Step 5: Merge on geo and year (left join from unemployment side)
merged_df = pd.merge(unemployment_yearly, df1_cleaned, on=['geo', 'year'], how='left')

# Optional: Rename columns for clarity
merged_df = merged_df.rename(columns={
    'unemployment': 'avg_unemployment_rate',
    'inflation': 'annual_inflation_rate'
})

merged_df.to_csv("merged_unemployment_inflation.csv", index=False)



In [6]:
import pandas as pd

# Step 1: Read the merged CSV file
df = pd.read_csv("merged_unemployment_inflation.csv")

# Step 2: Drop rows with any null values
df_cleaned = df.dropna()

# Step 3 (Optional): Check the result
print(df_cleaned.head())
print(f"Rows after dropping nulls: {len(df_cleaned)}")

# Optional: Save the cleaned version
df_cleaned.to_csv("merged_cleaned.csv", index=False)


   geo  year  avg_unemployment_rate freq       unit coicop    time  \
19  AT  2013              58.549537    A  RCH_A_AVG   CP00  2013.0   
20  AT  2014              62.037037    A  RCH_A_AVG   CP00  2014.0   
21  AT  2015              64.076389    A  RCH_A_AVG   CP00  2015.0   
22  AT  2016              68.285185    A  RCH_A_AVG   CP00  2016.0   
23  AT  2017              63.178549    A  RCH_A_AVG   CP00  2017.0   

    annual_inflation_rate  
19                    2.1  
20                    1.5  
21                    0.8  
22                    1.0  
23                    2.2  
Rows after dropping nulls: 415
